In [1]:
%%capture
!pip install git+https://github.com/pytorch/fairseq

In [2]:
!wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_small_960h.pt

--2022-05-04 23:51:27--  https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_small_960h.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 944029831 (900M) [application/octet-stream]
Saving to: ‘wav2vec_small_960h.pt’

wav2vec_small_960h. 100%[===================>] 900.30M  24.4MB/s    in 39s     

2022-05-04 23:52:07 (22.9 MB/s) - ‘wav2vec_small_960h.pt’ saved [944029831/944029831]



In [3]:
#!mkdir config
#!wget https://raw.githubusercontent.com/pytorch/fairseq/main/examples/wav2vec/config/finetuning/base_960h.yaml -O config/base_960h.yaml 

In [4]:
!mkdir config

In [5]:
%%writefile config/base_960h.yaml
common:
  fp16: true
  log_format: json
  log_interval: 200

checkpoint:
  no_epoch_checkpoints: true
  best_checkpoint_metric: wer
  restore_file: wav2vec_small_960h.pt

task:
  _name: audio_finetuning
  data: ???
  normalize: false
  labels: ltr

dataset:
  num_workers: 1
  max_tokens: 3200000
  skip_invalid_size_inputs_valid_test: true
  valid_subset: dev_other

distributed_training:
  ddp_backend: legacy_ddp
  distributed_world_size: 1

criterion:
  _name: ctc
  zero_infinity: true

optimization:
  max_update: 360000
  lr: [0.0001]
  sentence_avg: true

optimizer:
  _name: adam
  adam_betas: (0.9,0.98)
  adam_eps: 1e-08

lr_scheduler:
  _name: tri_stage
  phase_ratio: [0.1, 0.4, 0.5]
  final_lr_scale: 0.05

model:
  _name: wav2vec_ctc
  w2v_path: ???
  apply_mask: true
  mask_prob: 0.5
  mask_channel_prob: 0.1
  mask_channel_length: 64
  layerdrop: 0.1
  activation_dropout: 0.1
  feature_grad_mult: 0.0
  freeze_finetune_updates: 0

Writing config/base_960h.yaml


In [6]:
!fairseq-hydra-train task.data=../input/create-ljspeech-splits/10mins/ model.w2v_path=wav2vec_small_960h.pt --config-dir config --config-name base_960h

Traceback (most recent call last):
  File "/opt/conda/bin/fairseq-hydra-train", line 5, in <module>
    from fairseq_cli.hydra_train import cli_main
  File "/opt/conda/lib/python3.7/site-packages/fairseq_cli/hydra_train.py", line 15, in <module>
    from fairseq import distributed_utils, metrics
  File "/opt/conda/lib/python3.7/site-packages/fairseq/__init__.py", line 21, in <module>
    from fairseq.logging import meters, metrics, progress_bar  # noqa
  File "/opt/conda/lib/python3.7/site-packages/fairseq/logging/progress_bar.py", line 407, in <module>
    from torch.utils.tensorboard import SummaryWriter
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/tensorboard/__init__.py", line 4, in <module>
    LooseVersion = distutils.version.LooseVersion
AttributeError: module 'distutils' has no attribute 'version'
